# 1. Aquisicao e tratamento dos dados

## Importar bibliotecas

In [257]:
! pip install xlrd
! pip install openpyxl
! pip install pydantic
! pip install joblib

In [1]:
import pandas as pd
import os
import numpy as np
import yfinance as yf

from pydantic import BaseModel
from typing import Optional, List

import joblib

In [17]:
class AquisicaoDadosFundamentos(BaseModel):
    balancos_dir: Optional[str] = "../dados/balancos/"
    balancos_joblib_file: Optional[str] = "../dados/fundamentos.joblib"
    fund_by_code: Optional[dict] = {}
    codes: List[str] = []

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.codes = self.get_code_list()

    def run(self):
        if os.path.isfile(self.balancos_joblib_file):
            print("carregando joblib fundamentos")
            return joblib.load(self.balancos_joblib_file)

        self.get_balancos_by_code()
        self.get_dre_by_code()

        self.salvar_joblib()
        
        return self.fund_by_code
    
    def get_code_list(self):
        return [file.replace("balanco_", "").replace(".xls", "") for file in os.listdir(self.balancos_dir)]

    def get_balancos_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)

        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            print(code)
            balanco = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=0)
            # colocar codigo na posicao 0, 0
            balanco.iloc[0, 0] = code
            # mudar coluna
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            # tornar a 1ª coluna (que agora tem o nome da emrpesa)
            balanco = balanco.set_index(code)
            self.fund_by_code[code] = balanco

    def get_dre_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            dre = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=1)
            # na primeira coluna colocar o título com o nome da empresa
            dre.iloc[0, 0] = code
            # pegar 1ª linha e tornar um cabeçalho
            dre.columns = dre.iloc[0]
            dre = dre[1:]
            # tornar a 1ª coluna (que agora tem o nome da emrpesa)
            dre = dre.set_index(code)
            self.fund_by_code[code] = self.fund_by_code[code].append(dre)
    
        def salvar_joblib(self):
            joblib.dump(self.fund_by_code, self.balancos_joblib_file)



In [18]:
class AquisicaoDadosCotacoes(BaseModel):
    cotacoes_file: Optional[str] = "../dados/cotacoes/Cotacoes.xlsx"
    cotacoes_joblib_file: Optional[str] = "../dados/cotacoes.joblib"
    codes: List[str] = []
    cotacoes: Optional[dict] = {}

    def run(self):
        if os.path.isfile(self.cotacoes_joblib_file):
            print("carregando joblib cotacao")
            return joblib.load(self.cotacoes_joblib_file)

        self.get_cotacoes_from_excel()
        self.tratar_dados_nulos()
        self.salvar_joblib()
        return self.cotacoes

    def get_cotacoes_from_excel(self) -> None:
        """
        Le planilha com cotacoes historicas das acos do IBOV
        """
        
        cotacoes_df = pd.read_excel(self.cotacoes_file)

        for empresa in cotacoes_df['Empresa'].unique():
            self.cotacoes[empresa] = cotacoes_df.loc[cotacoes_df['Empresa']==empresa, :]
            
        print(len(self.cotacoes))

    def tratar_dados_nulos(self) -> None:
        """
        Remover empresas com cotacao contendo dados nulos
        """
        empresas_a_remover = []
        for empresa in self.codes:
            if self.cotacoes[empresa].isnull().values.any():
                self.cotacoes.pop(empresa)
                empresas_a_remover.append(empresa)
        self.codes = list(self.cotacoes.keys())
        print(len(self.codes))
        print(len(empresas_a_remover), empresas_a_remover, sep=" - ")

    def salvar_joblib(self):
        joblib.dump(self.cotacoes, self.cotacoes_joblib_file)



In [49]:
def remover_empresas_sem_dados_cotacao(fundamentos_by_code, cotacoes_by_code):
    fundamentos = fundamentos_by_code.copy()
    # remove fundamentos das empresas que tenham cotacoes com dados nulos
    codes_to_be_removed_from_fund = list(set(fundamentos.keys()) ^ set(cotacoes_by_code.keys()))
    # print(codes_to_be_removed_from_fund)
    for code in codes_to_be_removed_from_fund:
        fundamentos.pop(code)

    if cotacoes_by_code.keys() == fundamentos.keys():
        print("Empresas sem dados de cotacoes removidos com sucesso")
        # print(len(fundamentos.keys()))
    
    return fundamentos


In [53]:
def juntar_fundamentos_com_cotacoes(fundamentos_by_code, cotacoes_by_code):
    """
    Trata os data frames de fundamentos e junta as cotacoes por trimestre
    """
    fundamentos = fundamentos_by_code.copy()
    codes = fundamentos.keys()
    for code in codes:
        if "Adj Close" not in fundamentos[code].columns:
            df = fundamentos[code].T
            df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
            # print(df)

            # Definir data como indice e pegar somente coluna de Adj Close do df
            if code in cotacoes_by_code.keys():
                df_cotacao = cotacoes_by_code[code].set_index("Date")
                df_cotacao = df_cotacao[["Adj Close"]]

                # Juntar dois dataframes
                df = df.merge(df_cotacao, right_index=True, left_index=True)
                df.index.name = code

                fundamentos[code] = df

        else:
            print("Tratamento ja executado")
    
    print("dataframes juntados com sucesso - ", len(fundamentos))
    return fundamentos


### Ajeitando colunas

In [60]:

def remover_empresa_colunas_diff(fundamentos_by_code):

    fundamentos = fundamentos_by_code.copy()

    columns_ref = list(fundamentos["PETR4"].columns)
    """ 
    Remove empresas que nao tenham colunas de acordo com colunas de acao referencia 
    """
    codes = fundamentos.keys()
    empresa_a_remover = []
    for code in codes:
        if set(columns_ref) != set(fundamentos[code].columns):
            empresa_a_remover.append(code)
    
    for empresa in empresa_a_remover:
        fundamentos.pop(empresa)
    
    print(len(fundamentos), "empresas removidas por terem colunas diferentes de PETR4")

    return fundamentos




In [59]:
def remover_colunas_nulas(fundamentos_by_code):
    fundamentos = fundamentos_by_code.copy()
    codes = fundamentos.keys()

    colunas_remover = ["Receita Bruta de Vendas e/ou Serviços",
                        "Deduções da Receita Bruta",
                        "Resultado Não Operacional",
                        "Receitas",
                        "Despesas",
                        "Participações/Contribuições Estatutárias",
                        "Reversão dos Juros sobre Capital Próprio"]
    for code in codes:
        try:
            fundamentos[code] = fundamentos[code].drop(columns=colunas_remover, axis=1)
        except KeyError as e:
            print("df de empresas sem as colunas nulas", code)
            # raise e

    print("colunas nulas removidas com sucesso")
    return fundamentos

In [63]:
def remover_empresas_com_valores_nulos(fundamentos_by_code):
    
    fundamentos = fundamentos_by_code.copy()

    columns = fundamentos["PETR4"].columns

    valores_vazios = dict.fromkeys(columns, 0)
    total_linhas = 0
    empresa_with_null = []

    for empresa in fundamentos:
        tabela = fundamentos[empresa]
        total_linhas += tabela.shape[0]
        for coluna in columns:
            qtde_vazios = pd.isnull(tabela[coluna]).sum()
            if isinstance(qtde_vazios, np.int64):
                if qtde_vazios > 0:
                    # print(empresa, coluna, qtde_vazios, sep=" - ")
                    empresa_with_null.append(empresa)
                valores_vazios[coluna] += qtde_vazios

    # remove empresas com valores nulos, ignorando ABEV por ter nulo na ultima linha
    for empresa in set(empresa_with_null):
        if empresa != "ABEV3":
            print(f"empresa {empresa} removida por ter valores nulos")
            fundamentos.pop(empresa)
    
    return fundamentos

In [19]:
def get_ibov_from_yahoo():

    if os.path.isfile("../dados/cotacoes_ibov.joblib"):
        print("carregando joblib ibov")
        return joblib.load("../dados/cotacoes_ibov.joblib")

    data_inicial = "2012-12-20"
    data_final = "2021-09-20"

    ibov = yf.download("^BVSP", start=data_inicial, end=data_final)


In [67]:
def merge_ibov_by_fundamentos_dates(fundamentos_by_code, ibov):
    fundamentos = fundamentos_by_code.copy()
    datas_fundamentos = fundamentos["PETR4"].index

    # Set as nan when dates are different
    for data in datas_fundamentos:
        if data not in ibov.index:
            ibov.loc[data] = np.nan
    ibov = ibov.sort_index()
    ibov = ibov.ffill()
    ibov = ibov.rename(columns={"Adj Close": "IBOV"})

    for code in fundamentos:
        fundamentos[code] = fundamentos[code].merge(ibov[["IBOV"]], left_index=True, right_index=True)

    print(f"DF fundamentos e ibov juntados com sucesso - tamanho: {len(fundamentos)}")
    return fundamentos


In [72]:
def juntar_dataframes(fundamentos_by_code):
    copia_fund = fundamentos_by_code.copy()
    bd = pd.DataFrame()
    for code in copia_fund:
        copia_fund[code] = copia_fund[code][1:-1]
        copia_fund[code] = copia_fund[code].reset_index(drop=True)
        bd = bd.append(copia_fund[code])
    bd.reset_index(drop=True, inplace=True)
    print(f"Dataframe unico criado com sucesso. Registros: {len(bd)}")
    return bd


## Salvar dataframe em joblib

In [76]:
def salvar_df_in_joblib(df, file_name: str = "../out/database_unico.joblib"):

    joblib.dump(df, file_name)

    print("arquivo joblib de df unico criado com sucesso")


# Execucao

In [75]:
def main():

    # importar dados de balancos financeiros
    aquisicao_fund = AquisicaoDadosFundamentos()
    fundamentos_by_code = aquisicao_fund.run()

    # importar dados de cotacoes
    codes = list(fundamentos_by_code.keys())
    ad_cotacoes = AquisicaoDadosCotacoes(codes=codes)
    cotacoes_by_code = ad_cotacoes.run()

    fundamentos_by_code = remover_empresas_sem_dados_cotacao(fundamentos_by_code, cotacoes_by_code)
    fundamentos_by_code = juntar_fundamentos_com_cotacoes(fundamentos_by_code, cotacoes_by_code)


    # # tratar valores nulos e colunas
    fundamentos_by_code = remover_empresa_colunas_diff(fundamentos_by_code)
    fundamentos_by_code = remover_colunas_nulas(fundamentos_by_code)
    fundamentos_by_code = remover_empresas_com_valores_nulos(fundamentos_by_code)

    # # importar dados de cotacao do ibovespa
    ibov = get_ibov_from_yahoo()
    fundamentos_by_code = merge_ibov_by_fundamentos_dates(fundamentos_by_code, ibov)

    # # juntar os dataframes das empresas em um
    bd = juntar_dataframes(fundamentos_by_code)

    # # salvar em arquivo joblib
    salvar_df_in_joblib(bd)
    
    return bd
    
bd = main()

carregando joblib fundamentos
carregando joblib cotacao
Empresas sem dados de cotacoes removidos com sucesso
dataframes juntados com sucesso -  65
61 empresas removidas por terem colunas diferentes de PETR4
colunas nulas removidas com sucesso
empresa LCAM3 removida por ter valores nulos
empresa RADL3 removida por ter valores nulos
empresa CIEL3 removida por ter valores nulos
empresa CSAN3 removida por ter valores nulos
empresa SBSP3 removida por ter valores nulos
carregando joblib ibov
DF fundamentos e ibov juntados com sucesso - tamanho: 56
Dataframe unico criado com sucesso. Registros: 1728
